# 以創意擇優對新聞進行可信度評價

## 基本假設
1. 可信度定義：N(0, 1)，負數代表不可信，正數代表可信，0代表中立。
2. 每個使用者的評分都是理性的
3. 待補

另寫一篇文章，來講解這裡的東西，用小論文的格式

### 變數定義
變數名稱|中文|說明
:----|:----:|----
reporter|報導者|
news|新聞|
user|使用者|
review|評分|對新聞打分數
reviewer|評分者|對新聞打分數的使用者
judge|評價|對評分打分數
judger|評價者|對評分打分數的使用者
weight|權重|即為可信度

In [ ]:
# 初始化
import pandas as pd
import numpy as np

num = {
    'reporter' : 10,
    'news' : 150,
    'user' : 130,
    'review' : 150,
    'judge' : 200
}

# 評分分數範圍
class Scale:
    
    def __init__(self, min = 1, max = 5):
        self.min = min
        self.max = max
        self.mean = (min + max) / 2
        
    def arange(self):
        return np.arange(self.min, self.max + 1)

scale = Scale()

### 產生測試資料

In [ ]:
# 使用者
def get_users(num):
    users = pd.DataFrame({
        'user_weight' : np.repeat(scale.mean, num['user'])
    })
    return users

users = get_users(num)

users.head()

In [ ]:
# 報導者

def get_reporters(num):
    reporters = pd.DataFrame({
        'bvlty': np.random.randn(num['reporter'])
    })
    return reporters

reporters = get_reporters(num)

reporters.head()

In [ ]:
# 新聞
# X新聞可信度定義為：依據報導者的可信度產生的可信度 + 媒體偏好度
# 每個新聞都有一個真實的評分

def get_news(num, scale):
    
    # 先選擇每則新聞的報導者
    selected_reporters = np.random.choice(np.arange(num['reporter']), num['news'])

    # 找出每則新聞報導者的可信度
    # authors_bvlty = [reporters.iloc[author_index].bvlty for author_index in authors]

    # 定義每則新聞的可信度，因為與報導者相關，定義為 N(author.bvty, 1)，所以後面再加上報導者的可信度
    # news_bvlty = np.random.randn(num_news) + authors_bvlty

    # 每則新聞的真實評分
    # TODO 可以與報導者的素質做連結?
    news_real_scores = np.random.choice(scale.arange(), num['news'])

    # TODO 視覺化一下，是否可信度高的報導者會擁有可信度高的新聞

    # 新聞
    news = pd.DataFrame({
        'reporter_id' : selected_reporters,
        'news_real_score' : news_real_scores
    })
    
    return news

news = get_news(num, scale)

news.head()

In [ ]:
#使用者對新聞的評分
def get_reviews(num, scale):

    # 先選擇要被評分的新聞
    selected_news = np.random.choice(np.arange(num['news']), num['review'])

    # 使用者對該新聞的評分
    review_scores = np.random.choice(scale.arange(), num['review'])

    # 哪些使用者
    reviewers = np.random.choice(np.arange(num['user']), num['review'])

    reviews = pd.DataFrame({
        'reviewer_id' : reviewers,
        'news_id': selected_news,
        'review_score': review_scores
    })
    
    return reviews

reviews = get_reviews(num, scale)
reviews.head()

In [ ]:
#使用者對"其他使用者為新聞的評分"的評分

def get_judges(num, scale, reviews, news):
    
    # 先選擇要被評分的scoring
    selected_reviews = np.random.choice(np.arange(num['review']), num['judge'])

    # 取得新聞
    news_ids = [reviews.iloc[review_index].news_id for review_index in selected_reviews]

    # 取得這些新聞的真實評分
    news_real_scores = [news.iloc[news_index].news_real_score for news_index in news_ids]
    
    # 取得這些scoring(使用者對新聞)的評分
    review_scores = [reviews.iloc[review_index].review_score for review_index in selected_reviews]

    # 評價公式
    judging = lambda review_score, real_score: scale.max - abs(review_score - real_score)
    
    # 其他使用者對該scoring的評分(假設所有使用者都是理性的)
    judge_scores = [judging(review_scores[i], news_real_scores[i]) for i in range(num['judge'])]
    
    # 有哪些評價者
    judgers = np.random.choice(np.arange(num['user']), num['judge'])

    judges = pd.DataFrame({
        'review_id' : selected_reviews,
        'judger_id' : judgers,
        'judge_score': judge_scores
    })
    
    return judges

judges = get_judges(num, scale, reviews, news)

judges.head()

In [ ]:
# 合併成dataset
# data = 
data = judges.merge(reviews, left_on = 'review_id', right_index = True)
data = data.merge(news, left_on = 'news_id', right_index = True)
data = data.merge(users, left_on = 'reviewer_id', right_index = True).sort_index()

data.head()

In [ ]:
# 計算每個使用者的可信度/權重
# TODO 權重就是可信度嗎？ 可能是吧，權重越高，評價越重要

# 使用者的權重= (其他使用者對這個使用者的評分的評價 * 其他使用者的權重) 的 加權平均
# weight(user) = avg[score(judge_u1)*weight(user1) + score(judge_u2)*weight(user2) + ... + score(judge_uN)*weight(userN)]

# 事先計算每個judge的加權後分數 = 使用者對judge的評價 * (使用者的權重 / 權重的最大值)
data['judge_score_weighted'] = data['judge_score'] * data['user_weight'] / scale.max

# 計算每個使用者的新權重 = 每個judge的加權後分數 的平均
new_user_weight = data.groupby('reviewer_id', as_index = False)['judge_score_weighted'].mean()

# 更新data的使用者權重
new_user_weight.rename(columns = {'judge_score_weighted': 'user_weight'}, inplace = True)
data = data.drop(columns = ['user_weight'])
data = data.merge(new_user_weight, left_on = 'reviewer_id', right_on = 'reviewer_id',  how = 'left').sort_index()

data.head()

In [ ]:
# 計算新聞的可信度/權重

# 每個新聞的可信度等於 使用者對該新聞的評分 * 該使用者的可信度權重 的 加權平均。
# weight(news) = avg[score(review_u1)*weight(user1) + score(review_u2)*weight(user2) + ..... + score(review_uN)*weight(userN)]

# 先計算每個review的加權後分數 = 使用者對該新聞的評分 * (使用者的權重 / 權重的最大值)
data['review_score_weighted'] = data['review_score'] * data['user_weight'] / scale.max

# 計算每個新聞的可信度 / 權重
news_weight = data.groupby('news_id', as_index = False)['review_score_weighted'].mean()

# 在data加上新聞的可信度
news_weight.rename(columns = {'review_score_weighted': 'news_weight'}, inplace = True)
data = data.merge(news_weight, left_on = 'news_id', right_on = 'news_id',  how = 'left').sort_index()

data.head(10)

In [ ]:
# 計算記者的可信度 / 權重

# 每個記者的可信度來自於所寫新聞的可信度簡單平均數
# weight(reporter) = avg[weight(news1) + weight(news2) + .... + weight(newsN)]
reporter_weight = data.groupby('reporter_id', as_index = False)['news_weight'].mean()
reporter_weight.rename(columns = {'news_weight' : 'reporter_weight'}, inplace = True)
reporter_weight.head(10)